In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sayantikalaskar","key":"9c46cbc3dc0f3d53e9eef348359a288e"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d kondwani/eye-disease-dataset

Dataset URL: https://www.kaggle.com/datasets/kondwani/eye-disease-dataset
License(s): CC-BY-NC-SA-4.0
  0% 0.00/4.96M [00:00<?, ?B/s]
100% 4.96M/4.96M [00:00<00:00, 85.6MB/s]


In [6]:
!unzip eye-disease-dataset.zip

Archive:  eye-disease-dataset.zip
  inflating: Eye_diseases/Bulging_Eyes/image-10.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-11.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-12.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-13.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-14.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-15.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-2.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-3.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-4.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-5.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-6.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-7.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-8.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-9.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/images-10.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/images-11.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/i

In [7]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D,MaxPool2D
from tensorflow.keras.layers import Input, Dense
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from tensorflow.keras.applications.vgg16 import VGG16
import pandas as pd
import numpy as np
import os
from numpy import argmax

In [8]:
PATH = "/content/Eye_diseases"
data_dir_list = os.listdir(PATH)
data_dir_list

['Crossed_Eyes', 'Glaucoma', 'Cataracts', 'Uveitis', 'Bulging_Eyes']

In [9]:
img_rows=224
img_cols=224
num_channel=3

num_epoch = 50
batch_size = 32

img_data_list=[]
classes_names_list=[]
target_column=[]
for dataset in data_dir_list:
    classes_names_list.append(dataset)
    print("Getting image from {} folder".format(dataset))
    img_list= os.listdir(PATH +"/"+ dataset)
    for img in img_list:
        input_img = cv2.imread(PATH +"/"+dataset+"/"+img)
        input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
        img_data_list.append(input_img_resize)
        target_column.append(dataset)

Getting image from Crossed_Eyes folder
Getting image from Glaucoma folder
Getting image from Cataracts folder
Getting image from Uveitis folder
Getting image from Bulging_Eyes folder


In [10]:
num_classes = len(classes_names_list)
print("num_classes",num_classes)
img_data = np.array(img_data_list) # convert images in numpy array
img_data = img_data.astype('float32')
img_data /= 255
print("Shape of image data",img_data.shape)
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape
print("number of samples",num_of_samples)
print("target column before encoding",target_column)

num_classes 5
Shape of image data (383, 224, 224, 3)
number of samples 383
target column before encoding ['Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes'

In [11]:
Labelencoder = LabelEncoder()
target_column = Labelencoder.fit_transform(target_column)
np.unique(target_column)

array([0, 1, 2, 3, 4])

In [12]:
target_column

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

CNN Model

In [13]:
target_column_hotcoded = to_categorical(target_column,num_classes)
X,Y = shuffle(img_data,target_column_hotcoded,random_state=2)
X_train,X_temp,y_train,y_temp = train_test_split(X,Y,test_size=0.3,random_state=2)
X_test,X_val,y_test,y_val = train_test_split(X_temp,y_temp,test_size=0.3,random_state=2)

In [14]:
first_Mod = Sequential()

first_Mod.add(Conv2D(64,(3,3),activation='relu',input_shape=input_shape))
first_Mod.add(Conv2D(64,(3,3),activation='relu'))
first_Mod.add(MaxPool2D(pool_size=(2,2)))
first_Mod.add(Dropout(0.2))

first_Mod.add(Conv2D(64,(3,3),activation='relu'))
first_Mod.add(Conv2D(64,(3,3),activation='relu'))
first_Mod.add(MaxPool2D(pool_size=(2,2)))
first_Mod.add(Dropout(0.2))

first_Mod.add(Conv2D(128,(3,3),activation='relu'))
first_Mod.add(Conv2D(128,(3,3),activation='relu'))
first_Mod.add(MaxPool2D(pool_size=(2,2)))
first_Mod.add(Dropout(0.2))

first_Mod.add(Flatten())
first_Mod.add(Dense(128,activation='relu'))
first_Mod.add(Dropout(0.2))
first_Mod.add(Dense(128,activation='relu'))
first_Mod.add(Dropout(0.2))
first_Mod.add(Dense(num_classes,activation='softmax'))
first_Mod.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

first_Mod.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 64)      36928     
                                                        

In [15]:
hist = first_Mod.fit(X_train,y_train,batch_size=batch_size,epochs=num_epoch,verbose=1,validation_data=(X_test,y_test))
score = first_Mod.evaluate(X_test,y_test,batch_size=batch_size)
print('Test Loss',score[0])
print("Test Accuracy",score[1])

Epoch 1/50
9/9 [==============================] - 22s 953ms/step - loss: 1.5963 - accuracy: 0.3731 - val_loss: 1.4884 - val_accuracy: 0.4125
Epoch 2/50
9/9 [==============================] - 3s 290ms/step - loss: 1.4575 - accuracy: 0.4664 - val_loss: 1.4418 - val_accuracy: 0.4125
Epoch 3/50
9/9 [==============================] - 2s 264ms/step - loss: 1.4077 - accuracy: 0.4515 - val_loss: 1.4223 - val_accuracy: 0.4125
Epoch 4/50
9/9 [==============================] - 2s 267ms/step - loss: 1.3755 - accuracy: 0.4776 - val_loss: 1.4294 - val_accuracy: 0.4125
Epoch 5/50
9/9 [==============================] - 2s 265ms/step - loss: 1.4062 - accuracy: 0.4701 - val_loss: 1.5197 - val_accuracy: 0.4125
Epoch 6/50
9/9 [==============================] - 2s 264ms/step - loss: 1.4646 - accuracy: 0.4739 - val_loss: 1.4434 - val_accuracy: 0.4125
Epoch 7/50
9/9 [==============================] - 2s 264ms/step - loss: 1.4136 - accuracy: 0.4701 - val_loss: 1.4447 - val_accuracy: 0.4125
Epoch 8/50
9/9 [===

VGG16 Model

In [17]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the VGG16 model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the layers except the last 4 layers
for layer in vgg_base.layers[:-4]:
    layer.trainable = False

# Add custom layers on top of VGG16
x = vgg_base.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
vgg_model = Model(inputs=vgg_base.input, outputs=predictions)

# Compile the model
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
vgg_model.summary()

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Model checkpoint and early stopping
checkpoint = ModelCheckpoint('vgg_best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')

# Train the model
hist_vgg = vgg_model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                         epochs=num_epoch,
                         validation_data=(X_test, y_test),
                         callbacks=[checkpoint, early_stopping])

# Load the best model
vgg_model.load_weights('vgg_best_model.h5')

# Evaluate the model
score_vgg = vgg_model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test Loss (VGG16):', score_vgg[0])
print('Test Accuracy (VGG16):', score_vgg[1])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


9/9 [==============================] - 22s 1s/step - loss: 1.9833 - accuracy: 0.3507 - val_loss: 2.6494 - val_accuracy: 0.3875
Epoch 2/50
9/9 [==============================] - 4s 442ms/step - loss: 1.8466 - accuracy: 0.3918 - val_loss: 1.5862 - val_accuracy: 0.4125
Epoch 3/50
9/9 [==============================] - 6s 656ms/step - loss: 1.5818 - accuracy: 0.4739 - val_loss: 1.5769 - val_accuracy: 0.4125
Epoch 4/50
9/9 [==============================] - 4s 443ms/step - loss: 1.5496 - accuracy: 0.4701 - val_loss: 1.4707 - val_accuracy: 0.4125
Epoch 5/50
9/9 [==============================] - 5s 547ms/step - loss: 1.4900 - accuracy: 0.4478 - val_loss: 1.4744 - val_accuracy: 0.4125
Epoch 6/50
9/9 [==============================] - 4s 445ms/step - loss: 1.4470 - accuracy: 0.4701 - val_loss: 1.4372 - val_accuracy: 0.4125
Epoch 7/50
9/9 [==============================] - 4s 397ms/step - loss: 1.4173 - accuracy: 0.4739 - val_loss: 1.4692 - val_accuracy: 0.4125
Epoch 8/50
9/9 [=================

ResNet50 Model

In [18]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the ResNet50 model
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the layers except the last 4 layers
for layer in resnet_base.layers[:-4]:
    layer.trainable = False

# Add custom layers on top of ResNet50
x = resnet_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
resnet_model = Model(inputs=resnet_base.input, outputs=predictions)

# Compile the model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
resnet_model.summary()

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Model checkpoint and early stopping
checkpoint = ModelCheckpoint('resnet_best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')

# Train the model
hist_resnet = resnet_model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                               epochs=num_epoch,
                               validation_data=(X_test, y_test),
                               callbacks=[checkpoint, early_stopping])

# Load the best model
resnet_model.load_weights('resnet_best_model.h5')

# Evaluate the model
score_resnet = resnet_model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test Loss (ResNet50):', score_resnet[0])
print('Test Accuracy (ResNet50):', score_resnet[1])

94765736/94765736 [==============================] - 0s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                       